### Get Packages 

In [91]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import sklearn as sk
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
import datetime

%run ml_helpers.ipynb
%run ml.ipynb

               ID  Domestic  Beat  Year Month  Week  Day  Hour   Watch  PRCP  \
1117073  11621549     False   735  2019     3    11   13    12  Second  0.63   
931711   11365048     False  1533  2018     6    26   30    17   Third  0.00   
1160857  11687737     False   935  2019     5    20   14    17   Third  0.00   
837356   11239077     False  1012  2018     2     8   23    22   Third  0.20   
1291816  11879151     False  1931  2019    11    44    1    16   Third  0.00   
1269646  11846444      True   322  2019    10    40    1     1   First  0.00   
1154036  11676946     False   513  2019     5    18    5    12  Second  0.00   
882599   11303062     False   121  2018     4    18   30    12  Second  0.00   
977613   11426656     False  2515  2018     8    34   25    18   Third  0.11   
856741   11266189     False   131  2018     3    12   24    11  Second  0.00   
1062309  11539783      True  1012  2018    12    51   18    19   Third  0.00   
1100877  11596854     False  1114  2019 

Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2018 2017]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2017 2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2015 2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Fi

Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
         Beat  Year  Month   Watch  PRCP  SNOW  TMAX  TMIN  count_l_stops  \
1218199  1213  2019      7  Second  0.00   0.0    86    68            1.0   
1210037   624  2019      7   Third  0.00   0.0    92    75            0.0   
1210098   834  2019      7   Third  0.00   0.0    92    75            0.0   
1249274   522  2019      9   Third  0.17   0.0    81    68            0.0   
1174764  1512  2019      6  Second  0.21   0.0    82    54            2.0   
...       ...   ...    ...     ...   ...   ...   ...   ...            ...   
1253028  1032  2019      9   Third  0.05   0.0    67    62            0.0   
1281521  1622  2019     10  Second  0.00   0.0    56    41            0.0   
1080830   423  2019      1  Second  0.00   0.0    28    21            0.0   
1206822  1123  2019      7   Third  0.00   0.0    82    68            1.0   


Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2015]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [68]:
#Get Final data: 
data = pd.read_csv("../intermediate_data/high_crime_labeled.csv")
data = data.drop(columns=["beat", "beat_num", "Crimes", "Arrest", "Domestic", "Serious"])
data = convert_to_categorical(data, ["district", "sector",
                                    "Month",
                                     "Watch", "Beat"])
data.drop(columns = ["Unnamed: 0"], inplace=True)
data_list_20 = prep_data_test(data, "high_crime",
                                        2, "Year", ["district", "sector", 
                                                    "Month", 
                                                    "Beat", "Watch"])

      district sector  Year Month   Watch  Beat  count_l_stops  \
29557       17      1  2018     1   First  1713            3.0   
29558       17      1  2018     1  Second  1713            3.0   
29559       17      1  2018     1   Third  1713            3.0   
29560       17      1  2018     2   First  1713            3.0   
29561       17      1  2018     2  Second  1713            3.0   
...        ...    ...   ...   ...     ...   ...            ...   
49256        3      1  2019    11  Second   312            2.0   
49257        3      1  2019    11   Third   312            2.0   
49258        3      1  2019    12   First   312            2.0   
49259        3      1  2019    12  Second   312            2.0   
49260        3      1  2019    12   Third   312            2.0   

       count_bus_stops  count_metra_stops  count_restaurants  ...  \
29557             26.0                0.0               63.0  ...   
29558             26.0                0.0               63.0  ...   


Finished one-hot encoding...
Finished standardizing...
Working on: [2017 2018]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2016 2017]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2015 2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [86]:
def compute_scores_final_data(grid, model, model_name, data_list, target, target_year):
    '''
    Function that computes dataframe for results of best models
    Inputs:
        grid (dict): maps the model_name to a list which contains a dictionary mapping parameter (keys) to their 
                     values in the gest model
        model (sklearn): An sklearn algorithm
        model_name: a string assigned to the model, typically the same as model without the last parentheses
        data_list: list of tuples of dataframes. IN THIS CASE, this should include for 2020
        target (string): the column we seek to predict in the data frames
        target_year (int): 2020, the year we seek to predict
    Output:
        results (df): a dataframe with the results
    
    Note: the code that removes the params columns comes from here: https://stackoverflow.com/questions/19071199/drop-columns-whose-name-contains-a-specific-string-from-pandas-dataframe
    '''
    results = average_grid_searches(model, 
                                    model_name, grid, 
                                    data_list, 
                                    target, 
                                    target_year)
    results["model_type"]= model_name
    results["params"]=results["parameters_2020"]
    
    bool_array = results.columns.str.startswith("parameters")
    results2 = results.loc[:,~bool_array]
    return results2

## Logistic Regression

In [104]:
#Logistic Regression!
grid_log_reg ={"LogisticRegression": [{'penalty': 'l2',
                              'C': 0.001,
                              'random_state': 0}]}
results_log = compute_scores_final_data(grid_log_reg, LogisticRegression(),
                                    "LogisticRegression", data_list_20, "high_crime", 2020)

test year is: 2020
[2016, 2017, 2018, 2019]
[ True False  True False  True False  True False False False False]


## KNN

In [96]:
#KNN!
grid_knn = {"KNeighborsClassifier": [{'weights': 'uniform', 'n_neighbors': 10}]}
results_knn = compute_scores_final_data(grid_knn, KNeighborsClassifier(),
                                    "KNeighborsClassifier", data_list_20, "high_crime", 2020)

test year is: 2020
[2016, 2017, 2018, 2019]
[ True False  True False  True False  True False False False False]


## SVM: 

In [ ]:
grid_svm = { :[{}]}
results_svm = compute_scores_final_data()

## Naive Bayes: 

In [ ]:
grid_nb = { :[{}]}
results_nb = compute_scores_final_data()

## Decision Tree:

In [107]:
grid_dt = { :[{}]}
results_dt = compute_scores_final_data()

## Random Forest:

In [ ]:
grid_rf = { :[{}]}
results_rf = compute_scores_final_data()

## Aggregate Files:

In [106]:
results = pd.concat([results_log, results_knn
#                    , results_svm, results_nb, results_dt, results_rf
                    ])
results

,precision_2020,precision_2019,precision_2018,precision_2017,mean,model_type,params
0,0.626391,0.764843,0.782402,0.755897,0.732383,LogisticRegression,"{'penalty': 'l2', 'C': 0.001, 'random_state': 0}"
0,0.635498,0.793124,0.779119,0.795238,0.750745,KNeighborsClassifier,"{'weights': 'uniform', 'n_neighbors': 10}"
